In [ ]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Charger et préparer les données
data = pd.read_csv('../data/train_data.csv', sep=';')
X_train, X_val, y_train, y_val = train_test_split(data['text'], data['label'], test_size=0.2, random_state=2003)


In [18]:
# Charger le tokenizer CamemBERT
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
unique_labels = list(set(y_train))
print(unique_labels)
# Fonction pour tokeniser les données et créer un Dataset compatible
def tokenize_and_create_dataset(texts, labels, tokenizer, max_length=128):
    tokens = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_length)
    labels_encoded = [unique_labels.index(label) for label in labels]
    tokens['labels'] = labels_encoded
    return Dataset.from_dict(tokens)

# Créer les Datasets pour l'entraînement et la validation
train_dataset = tokenize_and_create_dataset(X_train, y_train, tokenizer)
val_dataset = tokenize_and_create_dataset(X_val, y_val, tokenizer)



['out_of_scope', 'book_flight', 'carry_on', 'flight_status', 'book_hotel', 'lost_luggage', 'translate', 'travel_alert', 'travel_suggestion']


In [10]:

# Charger le modèle CamemBERT pour la classification
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=len(set(y_train)))

# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",   # Évaluation par époque pour surveiller les performances
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,  # Garder uniquement le dernier checkpoint pour réduire l'espace de stockage
    save_strategy="epoch"  # Sauvegarde du modèle à chaque fin d'époque
)

# Préparer le Trainer avec les données et les labels
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Entraîner le modèle
trainer.train()


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/189 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 1.7302500009536743, 'eval_runtime': 21.0367, 'eval_samples_per_second': 11.979, 'eval_steps_per_second': 0.761, 'epoch': 1.0}


  0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 1.3672679662704468, 'eval_runtime': 17.9699, 'eval_samples_per_second': 14.023, 'eval_steps_per_second': 0.89, 'epoch': 2.0}


  0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 1.2680690288543701, 'eval_runtime': 23.6255, 'eval_samples_per_second': 10.666, 'eval_steps_per_second': 0.677, 'epoch': 3.0}
{'train_runtime': 1068.4368, 'train_samples_per_second': 2.819, 'train_steps_per_second': 0.177, 'train_loss': 1.6887061709449405, 'epoch': 3.0}


TrainOutput(global_step=189, training_loss=1.6887061709449405, metrics={'train_runtime': 1068.4368, 'train_samples_per_second': 2.819, 'train_steps_per_second': 0.177, 'total_flos': 52629629755824.0, 'train_loss': 1.6887061709449405, 'epoch': 3.0})

In [14]:
model.save_pretrained("fine_tuned_camembert_model")
tokenizer.save_pretrained("fine_tuned_camembert_model")
print("Modèle et tokenizer fine-tunés enregistrés avec succès.")

Modèle et tokenizer fine-tunés enregistrés avec succès.
